# Querying Structured Data

This notebook shows you how to build an agent that can answer questions based on structured data.
We take a CSV file, load it into a SQL-Lite database and then answer questions.
Of course, if you have a database, run the SQL on it directly.

## Set up.

Install the necessary packages, set up the API keys etc.

In [ ]:
%pip install --quiet -r requirements.txt

In [1]:
from dotenv import load_dotenv
load_dotenv("../keys.env");

PROVIDER = "Google"
#PROVIDER = "OpenAI"

if PROVIDER == "Google":
    from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
    model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.1)
else:
    from langchain_openai import ChatOpenAI, OpenAIEmbeddings
    model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)

## Download data about popular baby names

Download names data from the US Social Security administration and extract it to a CSV file

In [4]:
!wget --quiet https://www.ssa.gov/oact/babynames/names.zip

In [18]:
import zipfile, io

def concatenate_text_from_zip(zip_filename, output_filename):
    with zipfile.ZipFile(zip_filename, 'r') as zip_file:
        text_files = [name for name in zip_file.namelist() if name.endswith('.txt')]  # Identify text files
        with open(output_filename, 'w', encoding='utf-8') as output_file:
            output_file.write("name,gender,count,year\n")
            for filename in text_files:
                yob = filename.replace('yob','').replace('.txt','') # yob1979.txt -> 1979
                with zip_file.open(filename, 'r') as file:
                    for line in io.TextIOWrapper(file, 'utf-8'):
                        line = f"{line.strip()},{yob}\n"
                        output_file.write(line)
concatenate_text_from_zip("names.zip", "names_all_years.csv")

In [19]:
!head names_all_years.csv

name,gender,count,year
Mary,F,7065,1880
Anna,F,2604,1880
Emma,F,2003,1880
Elizabeth,F,1939,1880
Minnie,F,1746,1880
Margaret,F,1578,1880
Ida,F,1472,1880
Alice,F,1414,1880
Bertha,F,1320,1880


## Load the data into a SQL Lite database

This way, we can use SQL on the data.

In [20]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine
import pandas as pd

df = pd.read_csv("names_all_years.csv")
df.head()

,name,gender,count,year
0,Mary,F,7065,1880
1,Anna,F,2604,1880
2,Emma,F,2003,1880
3,Elizabeth,F,1939,1880
4,Minnie,F,1746,1880


In [21]:
engine = create_engine("sqlite:///names.db")
df.to_sql("names", engine, index=False)

2117219

Example of querying the data.  Let's find names in 1999 given to more than 20k babies.

In [25]:
db = SQLDatabase(engine=engine)
print(db.run("SELECT name, gender, count FROM names WHERE count > 20000 AND year == 1999;"))

[('Emily', 'F', 26541), ('Hannah', 'F', 21679), ('Jacob', 'M', 35370), ('Michael', 'M', 33913), ('Matthew', 'M', 30418), ('Joshua', 'M', 27261), ('Nicholas', 'M', 25639), ('Christopher', 'M', 25605), ('Andrew', 'M', 23864), ('Joseph', 'M', 23218), ('Daniel', 'M', 22704), ('Tyler', 'M', 22667), ('Brandon', 'M', 21602), ('Ryan', 'M', 21013), ('Austin', 'M', 20767), ('William', 'M', 20716), ('John', 'M', 20365), ('David', 'M', 20352)]


## Natural language to SQL

In [27]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(model, db=db, agent_type="openai-tools", verbose=True)

In [28]:
agent_executor.invoke({"input": "what was the most popular name for girls in 1999?"})

Key 'default' is not supported in schema, ignoring




> Entering new SQL Agent Executor chain...


Key 'default' is not supported in schema, ignoring



Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


names

Key 'default' is not supported in schema, ignoring


...

Key 'default' is not supported in schema, ignoring


```sql
SELECT name FROM names WHERE year = 1999 AND gender = 'F' ORDER BY count DESC LIMIT 1
```

Key 'default' is not supported in schema, ignoring



Invoking: `sql_db_query` with `{'query': "SELECT name FROM names WHERE year == 1999 AND gender == 'F' ORDER BY count DESC LIMIT 1"}`


[('Emily',)]The most popular name for girls in 1999 was Emily.

> Finished chain.


{'input': 'what was the most popular name for girls in 1999?',
 'output': 'The most popular name for girls in 1999 was Emily.'}

In [30]:
agent_executor.invoke({"input": "what was the most popular male name in the 1950s?"})

Key 'default' is not supported in schema, ignoring




> Entering new SQL Agent Executor chain...


Key 'default' is not supported in schema, ignoring



Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


names

Key 'default' is not supported in schema, ignoring


...

Key 'default' is not supported in schema, ignoring


```sql
SELECT name FROM names WHERE year BETWEEN 1950 AND 1959 AND gender = 'M' ORDER BY count DESC LIMIT 1
```

Key 'default' is not supported in schema, ignoring



Invoking: `sql_db_query` with `{'query': "SELECT name FROM names WHERE year BETWEEN 1950 AND 1959 AND gender = 'M' ORDER BY count DESC LIMIT 1"}`


[('Michael',)]The most popular male name in the 1950s was Michael. 


> Finished chain.


{'input': 'what was the most popular male name in the 1950s?',
 'output': 'The most popular male name in the 1950s was Michael. \n'}